In [2]:
# Import library untuk manipulasi data
import pandas as pd
import numpy as np

# Import library untuk preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Import library untuk menghitung cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

# Import library untuk visualisasi (opsional)
import matplotlib.pyplot as plt
import seaborn as sns

# Setting untuk display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Import Library yang Diperlukan

# Sistem Rekomendasi Lagu Berbasis Mood dan Energi

Notebook ini mengimplementasikan sistem rekomendasi lagu menggunakan metode **Content-Based Filtering** dengan algoritma **Cosine Similarity**.

## Tujuan:
- Merekomendasikan lagu berdasarkan input **Mood** dan **Level Energi** pengguna
- Menggunakan 5 fitur: Genre, Mood, Energy, Danceability, dan Tempo
- Menghitung kemiripan antar lagu menggunakan Cosine Similarity

## 2. Load Dataset

In [18]:
# Load dataset dari file CSV
df = pd.read_csv('music_sentiment_dataset.csv')

print(f"Dataset berhasil dimuat!")
print(f"Total baris: {len(df)}")
print(f"Total kolom: {len(df.columns)}")
print(f"\nPreview 5 baris pertama:")
df.head()

Dataset berhasil dimuat!
Total baris: 1000
Total kolom: 11

Preview 5 baris pertama:


,User_ID,User_Text,Sentiment_Label,Recommended_Song_ID,Song_Name,Artist,Genre,Tempo (BPM),Mood,Energy,Danceability
0,U1,Way ball purpose public experience recently re...,Sad,S1,Someone Like You,Adele,Pop,67,Melancholic,Low,Low
1,U2,Save officer two myself a.,Happy,S2,Happy,Pharrell Williams,Pop,160,Joyful,High,High
2,U3,Decade ahead everyone environment themselves a...,Relaxed,S3,Clair de Lune,Debussy,Classical,60,Soothing,Low,Low
3,U4,Best change letter citizen try ask quality pro...,Happy,S4,Happy,Pharrell Williams,Pop,160,Joyful,High,High
4,U5,Worker player chance kind actually.,Happy,S5,Happy,Pharrell Williams,Pop,160,Joyful,High,High


## 3. Explorasi Data Awal

In [4]:
# Informasi struktur dataset
print("Informasi Dataset:")
print(df.info())
print("\n" + "="*50 + "\n")

# Statistik deskriptif
print("Statistik Deskriptif:")
print(df.describe())
print("\n" + "="*50 + "\n")

# Cek missing values
print("Missing Values:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

# Cek jumlah lagu unik
unique_songs = df.drop_duplicates(subset=['Song_Name', 'Artist'])
print(f"Total lagu unik: {len(unique_songs)}")
print(f"Total entri dalam dataset: {len(df)}")

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   User_ID              1000 non-null   object
 1   User_Text            1000 non-null   object
 2   Sentiment_Label      1000 non-null   object
 3   Recommended_Song_ID  1000 non-null   object
 4   Song_Name            1000 non-null   object
 5   Artist               1000 non-null   object
 6   Genre                1000 non-null   object
 7   Tempo (BPM)          1000 non-null   int64 
 8   Mood                 1000 non-null   object
 9   Energy               1000 non-null   object
 10  Danceability         1000 non-null   object
dtypes: int64(1), object(10)
memory usage: 86.1+ KB
None


Statistik Deskriptif:
       Tempo (BPM)
count   1000.00000
mean      97.76000
std       36.74701
min       50.00000
25%       67.00000
50%      109.00000
75%      130.00000
max  

In [5]:
# Distribusi Sentiment Label
print("Distribusi Sentiment Label:")
print(df['Sentiment_Label'].value_counts())
print("\n" + "="*50 + "\n")

# Distribusi Energy Level
print("Distribusi Energy Level:")
print(df['Energy'].value_counts())
print("\n" + "="*50 + "\n")

# Distribusi Genre
print("Distribusi Genre:")
print(df['Genre'].value_counts())
print("\n" + "="*50 + "\n")

# Analisis lagu unik per mood
print("Analisis Lagu Unik per Sentiment:")
for sentiment in df['Sentiment_Label'].unique():
    sentiment_songs = unique_songs[unique_songs['Sentiment_Label'] == sentiment]
    print(f"  {sentiment}: {len(sentiment_songs)} lagu unik")
    print(f"    Energy distribution: {sentiment_songs['Energy'].value_counts().to_dict()}")

Distribusi Sentiment Label:
Sentiment_Label
Motivated    262
Happy        256
Sad          243
Relaxed      239
Name: count, dtype: int64


Distribusi Energy Level:
Energy
High    518
Low     482
Name: count, dtype: int64


Distribusi Genre:
Genre
Pop          263
Rock         253
Classical    134
Hip-Hop      132
Funk         113
Ambient      105
Name: count, dtype: int64


Analisis Lagu Unik per Sentiment:
  Sad: 2 lagu unik
    Energy distribution: {'Low': 2}
  Happy: 2 lagu unik
    Energy distribution: {'High': 2}
  Relaxed: 2 lagu unik
    Energy distribution: {'Low': 2}
  Motivated: 2 lagu unik
    Energy distribution: {'High': 2}


## 4. Preprocessing Data

In [6]:
# Buat copy dataframe untuk preprocessing
songs_df = df.copy()

# 1. Encoding fitur kategorikal menggunakan LabelEncoder
print("Melakukan encoding fitur kategorikal...")

# Inisialisasi LabelEncoder untuk setiap fitur
le_genre = LabelEncoder()
le_mood = LabelEncoder()
le_energy = LabelEncoder()
le_dance = LabelEncoder()

# Encoding setiap fitur
songs_df['Genre_Encoded'] = le_genre.fit_transform(songs_df['Genre'])
songs_df['Mood_Encoded'] = le_mood.fit_transform(songs_df['Mood'])
songs_df['Energy_Encoded'] = le_energy.fit_transform(songs_df['Energy'])
songs_df['Danceability_Encoded'] = le_dance.fit_transform(songs_df['Danceability'])

print("Encoding selesai!")
print("\nMapping Encoding:")
print(f"  Genre: {dict(zip(le_genre.classes_, le_genre.transform(le_genre.classes_)))}")
print(f"  Mood: {dict(zip(le_mood.classes_, le_mood.transform(le_mood.classes_)))}")
print(f"  Energy: {dict(zip(le_energy.classes_, le_energy.transform(le_energy.classes_)))}")
print(f"  Danceability: {dict(zip(le_dance.classes_, le_dance.transform(le_dance.classes_)))}")

Melakukan encoding fitur kategorikal...
Encoding selesai!

Mapping Encoding:
  Genre: {'Ambient': 0, 'Classical': 1, 'Funk': 2, 'Hip-Hop': 3, 'Pop': 4, 'Rock': 5}
  Mood: {'Calm': 0, 'Emotional': 1, 'Energetic': 2, 'Joyful': 3, 'Melancholic': 4, 'Powerful': 5, 'Soothing': 6}
  Energy: {'High': 0, 'Low': 1}
  Danceability: {'High': 0, 'Low': 1, 'Medium': 2}


In [7]:
# 2. Normalisasi fitur numerik (Tempo) menggunakan StandardScaler
print("\nMelakukan normalisasi fitur Tempo...")

scaler = StandardScaler()
songs_df['Tempo_Normalized'] = scaler.fit_transform(songs_df[['Tempo (BPM)']])

print("Normalisasi selesai!")
print(f"\nStatistik Tempo sebelum normalisasi:")
print(songs_df['Tempo (BPM)'].describe())
print(f"\nStatistik Tempo setelah normalisasi:")
print(songs_df['Tempo_Normalized'].describe())


Melakukan normalisasi fitur Tempo...
Normalisasi selesai!

Statistik Tempo sebelum normalisasi:
count    1000.00000
mean       97.76000
std        36.74701
min        50.00000
25%        67.00000
50%       109.00000
75%       130.00000
max       160.00000
Name: Tempo (BPM), dtype: float64

Statistik Tempo setelah normalisasi:
count    1.000000e+03
mean    -1.225686e-16
std      1.000500e+00
min     -1.300348e+00
25%     -8.374938e-01
50%      3.060283e-01
75%      8.777893e-01
max      1.694591e+00
Name: Tempo_Normalized, dtype: float64


In [8]:
# Preview hasil preprocessing
print("\nPreview data setelah preprocessing:")
display_cols = ['Song_Name', 'Artist', 'Genre_Encoded', 'Mood_Encoded', 
                'Energy_Encoded', 'Danceability_Encoded', 'Tempo_Normalized']
songs_df[display_cols].head(10)


Preview data setelah preprocessing:


,Song_Name,Artist,Genre_Encoded,Mood_Encoded,Energy_Encoded,Danceability_Encoded,Tempo_Normalized
0,Someone Like You,Adele,4,4,1,1,-0.837494
1,Happy,Pharrell Williams,4,3,0,0,1.694591
2,Clair de Lune,Debussy,1,6,1,1,-1.028081
3,Happy,Pharrell Williams,4,3,0,0,1.694591
4,Happy,Pharrell Williams,4,3,0,0,1.694591
5,Eye of the Tiger,Survivor,5,2,0,2,0.306028
6,Someone Like You,Adele,4,4,1,1,-0.837494
7,Someone Like You,Adele,4,4,1,1,-0.837494
8,Eye of the Tiger,Survivor,5,2,0,2,0.306028
9,Fix You,Coldplay,5,1,1,2,-0.619680


## 5. Membuat Feature Matrix dan Menghitung Cosine Similarity

In [9]:
# Membuat feature matrix dengan 5 fitur
print("Membuat feature matrix...")

# Kolom fitur yang akan digunakan untuk menghitung similarity
feature_columns = ['Genre_Encoded', 'Mood_Encoded', 'Energy_Encoded', 
                   'Danceability_Encoded', 'Tempo_Normalized']

# Ekstrak nilai dari kolom fitur menjadi array NumPy
feature_matrix = songs_df[feature_columns].values

print(f"Feature matrix berhasil dibuat!")
print(f"Dimensi feature matrix: {feature_matrix.shape}")
print(f"   - Jumlah lagu: {feature_matrix.shape[0]}")
print(f"   - Jumlah fitur: {feature_matrix.shape[1]}")
print(f"\nContoh 5 baris pertama feature matrix:")
print(feature_matrix[:5])

Membuat feature matrix...
Feature matrix berhasil dibuat!
Dimensi feature matrix: (1000, 5)
   - Jumlah lagu: 1000
   - Jumlah fitur: 5

Contoh 5 baris pertama feature matrix:
[[ 4.          4.          1.          1.         -0.83749376]
 [ 4.          3.          0.          0.          1.69459075]
 [ 1.          6.          1.          1.         -1.02808077]
 [ 4.          3.          0.          0.          1.69459075]
 [ 4.          3.          0.          0.          1.69459075]]


In [10]:
# Menghitung Cosine Similarity Matrix
print("\nMenghitung Cosine Similarity Matrix...")
print("   Formula: cosine_similarity(A,B) = (A · B) / (||A|| × ||B||)")

# Hitung similarity matrix (ukuran: jumlah_lagu × jumlah_lagu)
similarity_matrix = cosine_similarity(feature_matrix)

print(f"\nCosine Similarity Matrix berhasil dihitung!")
print(f"Dimensi similarity matrix: {similarity_matrix.shape}")
print(f"   - Setiap lagu dibandingkan dengan {similarity_matrix.shape[1]} lagu")
print(f"\nStatistik nilai similarity:")
print(f"   - Min: {similarity_matrix.min():.4f}")
print(f"   - Max: {similarity_matrix.max():.4f}")
print(f"   - Mean: {similarity_matrix.mean():.4f}")
print(f"   - Median: {np.median(similarity_matrix):.4f}")


Menghitung Cosine Similarity Matrix...
   Formula: cosine_similarity(A,B) = (A · B) / (||A|| × ||B||)

Cosine Similarity Matrix berhasil dihitung!
Dimensi similarity matrix: (1000, 1000)
   - Setiap lagu dibandingkan dengan 1000 lagu

Statistik nilai similarity:
   - Min: -0.2173
   - Max: 1.0000
   - Mean: 0.6854
   - Median: 0.8339


## 6. Implementasi Fungsi Rekomendasi

In [11]:
def get_recommendations(user_mood, user_energy, top_n=10):
    """
    Fungsi untuk mendapatkan rekomendasi lagu berdasarkan mood dan energi pengguna
    
    Parameter:
        user_mood (str): Mood pengguna (Happy, Sad, Relaxed, Motivated)
        user_energy (str): Level energi (High, Medium, Low)
        top_n (int): Jumlah rekomendasi yang diinginkan (default: 10)
    
    Return:
        list: List berisi dictionary informasi lagu yang direkomendasikan
    """
    
    # LANGKAH 1: Filter berdasarkan Sentiment_Label
    mood_songs = songs_df[songs_df['Sentiment_Label'].str.lower() == user_mood.lower()].copy()
    
    # Jika tidak ada hasil, coba mapping ke Mood spesifik
    if mood_songs.empty:
        sentiment_to_mood = {
            'happy': ['joyful', 'energetic'],
            'sad': ['melancholic', 'emotional'],
            'relaxed': ['soothing', 'calm'],
            'motivated': ['energetic', 'powerful']
        }
        user_mood_lower = user_mood.lower()
        target_moods = sentiment_to_mood.get(user_mood_lower, [user_mood_lower])
        mood_songs = songs_df[songs_df['Mood'].str.lower().isin(target_moods)].copy()
    
    if mood_songs.empty:
        return []
    
    # LANGKAH 2: Filter berdasarkan Energy
    if user_energy:
        energy_filtered = mood_songs[mood_songs['Energy'].str.lower() == user_energy.lower()]
        if not energy_filtered.empty:
            mood_songs = energy_filtered
    
    if mood_songs.empty:
        return []
    
    # LANGKAH 3: Hapus duplikasi berdasarkan Song_Name + Artist
    unique_songs = mood_songs.drop_duplicates(subset=['Song_Name', 'Artist'], keep='first')
    
    # LANGKAH 4: Dapatkan index lagu
    song_indices = unique_songs.index.tolist()
    
    # LANGKAH 5: Shuffle untuk variasi
    np.random.shuffle(song_indices)
    
    # LANGKAH 6: Ranking berdasarkan similarity
    if len(song_indices) > top_n:
        similarity_scores = []
        
        for idx in song_indices:
            # Hitung rata-rata similarity dengan lagu lain
            scores = similarity_matrix[idx]
            avg_score = np.mean(scores)
            
            # Tambahkan random factor untuk variasi (0.85 - 1.15)
            random_factor = np.random.uniform(0.85, 1.15)
            similarity_scores.append((idx, avg_score * random_factor))
        
        # Urutkan berdasarkan score tertinggi
        similarity_scores.sort(key=lambda x: x[1], reverse=True)
        top_indices = [i[0] for i in similarity_scores[:top_n]]
    else:
        top_indices = song_indices[:top_n]
    
    # LANGKAH 7: Buat list rekomendasi
    recommendations = []
    seen_songs = set()
    
    for idx in top_indices:
        song = songs_df.iloc[idx]
        song_key = f"{song['Song_Name']}_{song['Artist']}"
        
        if song_key not in seen_songs:
            seen_songs.add(song_key)
            recommendations.append({
                'Song_Name': song['Song_Name'],
                'Artist': song['Artist'],
                'Genre': song['Genre'],
                'Mood': song['Mood'],
                'Energy': song['Energy'],
                'Tempo': song['Tempo (BPM)'],
                'Danceability': song['Danceability']
            })
    
    return recommendations

print("Fungsi rekomendasi berhasil dibuat!")

Fungsi rekomendasi berhasil dibuat!


## 7. Testing Sistem Rekomendasi

Sekarang kita akan menguji sistem rekomendasi dengan berbagai kombinasi mood dan energi.

In [12]:
# Test 1: Sad + Low Energy
print("="*70)
print("TEST 1: Mood = Sad, Energy = Low")
print("="*70)

recommendations = get_recommendations(user_mood='Sad', user_energy='Low', top_n=10)

if recommendations:
    print(f"\nDitemukan {len(recommendations)} rekomendasi lagu:\n")
    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['Song_Name']} - {song['Artist']}")
        print(f"   Genre: {song['Genre']} | Mood: {song['Mood']} | Energy: {song['Energy']}")
        print(f"   Tempo: {song['Tempo']} BPM | Danceability: {song['Danceability']}\n")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

TEST 1: Mood = Sad, Energy = Low

Ditemukan 2 rekomendasi lagu:

1. Fix You - Coldplay
   Genre: Rock | Mood: Emotional | Energy: Low
   Tempo: 75 BPM | Danceability: Medium

2. Someone Like You - Adele
   Genre: Pop | Mood: Melancholic | Energy: Low
   Tempo: 67 BPM | Danceability: Low



In [13]:
# Test 2: Happy + High Energy
print("="*70)
print("TEST 2: Mood = Happy, Energy = High")
print("="*70)

recommendations = get_recommendations(user_mood='Happy', user_energy='High', top_n=10)

if recommendations:
    print(f"\nDitemukan {len(recommendations)} rekomendasi lagu:\n")
    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['Song_Name']} - {song['Artist']}")
        print(f"   Genre: {song['Genre']} | Mood: {song['Mood']} | Energy: {song['Energy']}")
        print(f"   Tempo: {song['Tempo']} BPM | Danceability: {song['Danceability']}\n")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

TEST 2: Mood = Happy, Energy = High

Ditemukan 2 rekomendasi lagu:

1. Uptown Funk - Bruno Mars
   Genre: Funk | Mood: Energetic | Energy: High
   Tempo: 115 BPM | Danceability: High

2. Happy - Pharrell Williams
   Genre: Pop | Mood: Joyful | Energy: High
   Tempo: 160 BPM | Danceability: High


TEST 2: Mood = Happy, Energy = High

Ditemukan 2 rekomendasi lagu:

1. Uptown Funk - Bruno Mars
   Genre: Funk | Mood: Energetic | Energy: High
   Tempo: 115 BPM | Danceability: High

2. Happy - Pharrell Williams
   Genre: Pop | Mood: Joyful | Energy: High
   Tempo: 160 BPM | Danceability: High



In [14]:
# Test 3: Relaxed + Low Energy
print("="*70)
print("TEST 3: Mood = Relaxed, Energy = Low")
print("="*70)

recommendations = get_recommendations(user_mood='Relaxed', user_energy='Low', top_n=10)

if recommendations:
    print(f"\nDitemukan {len(recommendations)} rekomendasi lagu:\n")
    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['Song_Name']} - {song['Artist']}")
        print(f"   Genre: {song['Genre']} | Mood: {song['Mood']} | Energy: {song['Energy']}")
        print(f"   Tempo: {song['Tempo']} BPM | Danceability: {song['Danceability']}\n")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

TEST 3: Mood = Relaxed, Energy = Low

Ditemukan 2 rekomendasi lagu:

1. Weightless - Marconi Union
   Genre: Ambient | Mood: Calm | Energy: Low
   Tempo: 50 BPM | Danceability: Low

2. Clair de Lune - Debussy
   Genre: Classical | Mood: Soothing | Energy: Low
   Tempo: 60 BPM | Danceability: Low



In [15]:
# Test 4: Motivated + High Energy
print("="*70)
print("TEST 4: Mood = Motivated, Energy = High")
print("="*70)

recommendations = get_recommendations(user_mood='Motivated', user_energy='High', top_n=10)

if recommendations:
    print(f"\nDitemukan {len(recommendations)} rekomendasi lagu:\n")
    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['Song_Name']} - {song['Artist']}")
        print(f"   Genre: {song['Genre']} | Mood: {song['Mood']} | Energy: {song['Energy']}")
        print(f"   Tempo: {song['Tempo']} BPM | Danceability: {song['Danceability']}\n")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

TEST 4: Mood = Motivated, Energy = High

Ditemukan 2 rekomendasi lagu:

1. Stronger - Kanye West
   Genre: Hip-Hop | Mood: Powerful | Energy: High
   Tempo: 130 BPM | Danceability: High

2. Eye of the Tiger - Survivor
   Genre: Rock | Mood: Energetic | Energy: High
   Tempo: 109 BPM | Danceability: Medium


TEST 4: Mood = Motivated, Energy = High

Ditemukan 2 rekomendasi lagu:

1. Stronger - Kanye West
   Genre: Hip-Hop | Mood: Powerful | Energy: High
   Tempo: 130 BPM | Danceability: High

2. Eye of the Tiger - Survivor
   Genre: Rock | Mood: Energetic | Energy: High
   Tempo: 109 BPM | Danceability: Medium



In [16]:
# Test 5: Sad + High Energy (untuk melihat perilaku fallback)
print("="*70)
print("TEST 5: Mood = Sad, Energy = High")
print("="*70)
print("Catatan: Kombinasi ini mungkin tidak tersedia di dataset.")
print("   Sistem akan menggunakan fallback ke lagu Sad dengan energi apapun.\n")

recommendations = get_recommendations(user_mood='Sad', user_energy='High', top_n=10)

if recommendations:
    print(f"\nDitemukan {len(recommendations)} rekomendasi lagu:\n")
    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['Song_Name']} - {song['Artist']}")
        print(f"   Genre: {song['Genre']} | Mood: {song['Mood']} | Energy: {song['Energy']}")
        print(f"   Tempo: {song['Tempo']} BPM | Danceability: {song['Danceability']}\n")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

TEST 5: Mood = Sad, Energy = High
Catatan: Kombinasi ini mungkin tidak tersedia di dataset.
   Sistem akan menggunakan fallback ke lagu Sad dengan energi apapun.


Ditemukan 2 rekomendasi lagu:

1. Fix You - Coldplay
   Genre: Rock | Mood: Emotional | Energy: Low
   Tempo: 75 BPM | Danceability: Medium

2. Someone Like You - Adele
   Genre: Pop | Mood: Melancholic | Energy: Low
   Tempo: 67 BPM | Danceability: Low



## 8. Analisis Hasil

Mari kita analisis karakteristik sistem rekomendasi yang telah dibuat.

In [17]:
# Fungsi helper untuk menganalisis rekomendasi
def analyze_recommendations():
    """
    Fungsi untuk menganalisis ketersediaan kombinasi mood-energi dalam dataset
    """
    print("ANALISIS KETERSEDIAAN KOMBINASI MOOD-ENERGI\n")
    print("="*70)
    
    moods = ['Happy', 'Sad', 'Relaxed', 'Motivated']
    energies = ['High', 'Medium', 'Low']
    
    results = []
    
    for mood in moods:
        for energy in energies:
            recs = get_recommendations(mood, energy, top_n=10)
            count = len(recs)
            results.append({
                'Mood': mood,
                'Energy': energy,
                'Jumlah Rekomendasi': count,
                'Status': 'Tersedia' if count > 0 else 'Tidak Ada'
            })
    
    # Buat DataFrame untuk visualisasi
    df_results = pd.DataFrame(results)
    
    print("\nTabel Ketersediaan Kombinasi:\n")
    print(df_results.to_string(index=False))
    
    # Statistik
    print("\n" + "="*70)
    print("\nSTATISTIK:")
    total_combinations = len(results)
    available = len([r for r in results if r['Jumlah Rekomendasi'] > 0])
    print(f"   Total kombinasi: {total_combinations}")
    print(f"   Kombinasi tersedia: {available}")
    print(f"   Kombinasi tidak tersedia: {total_combinations - available}")
    print(f"   Persentase ketersediaan: {(available/total_combinations)*100:.1f}%")
    
    return df_results

# Jalankan analisis
df_analysis = analyze_recommendations()

ANALISIS KETERSEDIAAN KOMBINASI MOOD-ENERGI


Tabel Ketersediaan Kombinasi:

     Mood Energy  Jumlah Rekomendasi   Status
    Happy   High                   2 Tersedia
    Happy Medium                   2 Tersedia
    Happy    Low                   2 Tersedia
      Sad   High                   2 Tersedia
      Sad Medium                   2 Tersedia
      Sad    Low                   2 Tersedia
  Relaxed   High                   2 Tersedia
  Relaxed Medium                   2 Tersedia
  Relaxed    Low                   2 Tersedia
Motivated   High                   2 Tersedia
Motivated Medium                   2 Tersedia
Motivated    Low                   2 Tersedia


STATISTIK:
   Total kombinasi: 12
   Kombinasi tersedia: 12
   Kombinasi tidak tersedia: 0
   Persentase ketersediaan: 100.0%


Tabel Ketersediaan Kombinasi:

     Mood Energy  Jumlah Rekomendasi   Status
    Happy   High                   2 Tersedia
    Happy Medium                   2 Tersedia
    Happy    Low          

## 9. Kesimpulan

### Hasil yang Dicapai:
1. Sistem berhasil mengimplementasikan **Content-Based Filtering** dengan **Cosine Similarity**
2. Menggunakan **5 fitur** untuk perhitungan kemiripan: Genre, Mood, Energy, Danceability, Tempo
3. Sistem dapat memberikan rekomendasi berdasarkan input **Mood** dan **Level Energi**
4. Menerapkan **penghapusan duplikasi** untuk memastikan setiap lagu unik hanya muncul sekali
5. Menggunakan **random factor** untuk variasi hasil rekomendasi

### Keterbatasan:
1. Dataset hanya memiliki **8 lagu unik** sehingga variasi rekomendasi terbatas
2. Beberapa kombinasi mood-energi tidak tersedia (misal: Sad + High Energy)
3. Jumlah rekomendasi bisa kurang dari target jika kandidat terbatas

### Saran Pengembangan:
1. Perluas dataset dengan lebih banyak lagu unik
2. Tambahkan fitur analisis emosi dari lirik lagu
3. Implementasi strategi fallback yang lebih adaptif
4. Tambahkan evaluasi kuantitatif (Precision, Recall, F1-Score)
5. Integrasi dengan API musik untuk data real-time